# _Features_ Cuello de Botella

Los features cuello de botella son la representación de las imágenes pasada por las capas inmutables (congeladas) de la red.

Pre-calcular estos features usando modelos pre-entrenados es una gran manera de ahorrar tiempo y, a su vez, de utilizar modelos o algoritmos más tradicionales como MLPs, SVCs e incluso decision trees.

En este notebook calcularemos un par de features cuello de botella usando VGG16.

## Cargando y Pre-procesando una Muestra de Imágenes

Calcular features cuello de botella puede ser bastante costoso, tanto a nivel de tiempo como de poder de cómputo.

Por esta razón, sólo tomaremos un pequeño subconjunto de las imágenes que usamos en el notebook previo.

In [1]:
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
import numpy as np
from glob import glob
import random

SAMPLE_SIZE = 50

image_paths = glob('dog_images/*/*/*')
image_paths = random.sample(image_paths, SAMPLE_SIZE)

def path_to_tensor(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    x = image.img_to_array(img)
    
    return np.expand_dims(x, axis=0)

def paths_to_tensor(image_paths):
    tensors = [path_to_tensor(image_path) for image_path in image_paths]
    return np.vstack(tensors)

# Calculate the image input
image_input = preprocess_input(paths_to_tensor(image_paths))

print(image_input.shape)

Using TensorFlow backend.


(50, 224, 224, 3)


## Importando VGG16

Ahora podemos importar VGG16 sin las capas totalmente conectadas.

In [2]:
from keras.applications.vgg16 import VGG16
model = VGG16(include_top=False)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

## Creando Cuellos de Botella

Como pudimos ver anteriormente, la última capa de esta red es de max pooling. Esto nos dará como resultado un tensor tridimensional. Creemos los cuellos de botella:

In [3]:
bottlenecked_batch = model.predict(image_input)
print(f'Bottlenecked batch shape: {bottlenecked_batch.shape}')

Bottlenecked batch shape: (50, 7, 7, 512)
